# A Simple Fuzzer

In [57]:
from typing import Dict, List, Tuple, Union, Any

In [52]:
import random

In [53]:
def fuzzer(max_length: int = 100, char_start: int = 32, char_range: int = 32) -> str:
    '''
    A string of up to max_length characters
    int the range [char_start, char_start + char_range) 
    '''
    str_len = random.randrange(0, max_length + 1)
    out = ''
    for i in range(0, str_len):
        out += chr(random.randrange(char_start, char_start + char_range))
    return out

In [54]:
fuzzer()

'1+#</+ >""-$#)5\'/)$ 9&35&6?1'

In [55]:
fuzzer(1000, ord('a'), 26)  # produce massive a~z data string

'xuwohcfunwmvzgahpegivwryaghavqptyqpavqgxyzmihaoyvyyjtqulaldrpgtttbqkscgrjsxcyzocizlqvahhqjrmwudiyjlogfbqihtpmeuqlgxlrjkrsbqnhfrpkohyqxztosovkpmtqgamgwnkivxasmdphehjpxgcpcodtrmbxtwtzzecgkdllkwxapiadwowssiotifiqzytuqxkiuwuqrdzrtqkpdpdggsddrfuadigcyisfpffhqidguhrgovmxdbnlsmzoevinoytmvzdughugkstaaeudynlxewrchkctorwnjrsutvcqokqdamnzgmnmyoblliutjhigzwotlmtnhqdstuorkrtdrozdlzspykughkprrrjmvvmgbrxmfmwkbearjogkzgelgosnonqoefyiqlkwaybknzewhwzuxqzojmjazxhhkiiysomxaynqeblxjdledfaxffmpxdlcknhypqbfrpzqmxseiqpjakmavhumcrxvhbzmeuutbxeyqyzraipjtvxofkhgxbimvdvtyscopuanoutkwwgnzqexabckklscbfmowqazpxgfxledtflncexvcwopggswyvnqbbzdlrhphyxzupbxhjwjwy'

In [60]:
fuzzer(200, ord('0'), 10)

'622201427167862456189066761579192143889562702599405693662225688163355487710556490727748873145676573951098158581119265'

# Fuzzing External Programs

## Creating Input Files

In [61]:
import os
import tempfile

In [62]:
basename = 'input.txt'
tempdir = tempfile.mkdtemp()
FILE = os.path.join(tempdir, basename)
print(FILE)

C:\Users\hxie\AppData\Local\Temp\tmpad8upzjf\input.txt


In [64]:
data = fuzzer()
with open(FILE, 'w') as f:
    f.write(data)

In [66]:
contents = open(FILE).read()
print(contents)
assert(contents == data)

0!9)="=:/02=


## Invoking External Programs

In [68]:
import os
import subprocess

bc is a math command in Unix

In [73]:
program = 'C:\\Windows\\System32\\calc.exe'
with open(FILE, 'w') as f:
    f.write('2 + 2\n')
result = subprocess.run([program, FILE],
                        stdin=subprocess.DEVNULL,
                        stdout=subprocess.PIPE,
                        stderr=subprocess.PIPE,
                        universal_newlines=True)
print(result)

CompletedProcess(args=['C:\\Windows\\System32\\calc.exe', 'C:\\Users\\hxie\\AppData\\Local\\Temp\\tmpad8upzjf\\input.txt'], returncode=0, stdout='', stderr='')


In [74]:
result.stdout

''

In [75]:
result.stderr

''

In [76]:
result.returncode

0

# A Fuzzing Architecture

## Runner Classes

In [81]:
class Runner:
    '''
    Base class for testing inputs
    '''
    PASS = 'PASS'
    FAIL = 'FAIL'
    UNRESOLVED = 'UNRESOLVED'

    def __init__(self) -> None:
        pass
        
    def run(self, inp: str) -> Any:
        return(inp, Runner.UNRESOLVED)

In [82]:
class PrintRunner(Runner):
    def run(self, inp) -> Any:
        print(inp)
        return(inp, Runner.UNRESOLVED)

In [83]:
p = PrintRunner()
(result, outcome) = p.run('Some input')

Some input


In [84]:
result

'Some input'

In [85]:
outcome

'UNRESOLVED'

## Fuzzer Classes

In [89]:
Outcome = str

In [92]:
class Fuzzer:
    '''Base class for fuzzers'''
    def __init__(self) -> None:
        pass

    def fuzz(self) -> str:
        return ''
    
    def run(self, runner: Runner = Runner()) \
        -> Tuple[subprocess.CompletedProcess, Outcome]:
        return runner.run(self.fuzz())
    
    def runs(self, runner: Runner = PrintRunner(), trails: int = 10) \
        -> List[Tuple[subprocess.CompletedProcess, Outcome]]:
        return [self.run(runner) for i in range(trails)]

In [93]:
class RandomFuzzer(Fuzzer):
    """Produce random inputs."""

    def __init__(self, min_length: int = 10, max_length: int = 100,
                 char_start: int = 32, char_range: int = 32) -> None:
        """Produce strings of `min_length` to `max_length` characters
           in the range [`char_start`, `char_start` + `char_range`)"""
        self.min_length = min_length
        self.max_length = max_length
        self.char_start = char_start
        self.char_range = char_range

    def fuzz(self) -> str:
        string_length = random.randrange(self.min_length, self.max_length + 1)
        out = ""
        for i in range(0, string_length):
            out += chr(random.randrange(self.char_start,
                                        self.char_start + self.char_range))
        return out

In [94]:
random_fuzzer = RandomFuzzer(min_length=20, max_length=20)
for i in range(10):
    print(random_fuzzer.fuzz())

5&2&3: 5!(!19#..10*6
6<*8?5+8'40(1;: ?=5<
+)>4<"'(2/293&7:9)0?
8-/!3&*580**82,!%6%;
68 <,".('%;<32"39/</
7/(?52=&:%16'&(=,;&%
"9:;+34&70'#*%84<::1
+;5>!*3>6<31-.%&'-/)
:1=9)8830"$- 39$*7(.
*;3*63?,67869#5:2,#7
